In [ ]:
%reset

In [ ]:
import numpy as np
import random
import math
import torch
import torch.nn as nn
import torch.nn.functional as func
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from DataAccessAndFormatting import DataFormatting
from DataAccessAndFormatting import GaussianKernelFiltering

In [ ]:
'''
    Global variables
    path                : local path to the datasets
    index_database      : enables access to the datasets of individual moths
                            the dataset for subject (moth) i is stored as .mat file with name format 
                            "Moth{index_database[0][i]}_{index_database[1][i]}__AnnotatedToShare_v3"
    all_neurons         : channel (muscle) codes
    forces              : force/torque codes
    stimuli             : stimuli notation
    stimuli_names       : stimuli codes
'''

path = "../../Data/Moth"
index_database = [ [1       , 2       , 3       , 4       , 5       , 6       , 7       , 8       , 9       ,       10], 
                   [20200113, 20200114, 20200219, 20200221, 20200226, 20200818, 20200819, 20200915, 20201006, 20201013] ]
all_neurons = ['LAX','LBA','LDLM','LDVM','LSA',  'RAX','RBA','RDLM','RDVM','RSA']
forces = ['Fx_by_WS','Fy_by_WS','Fz_by_WS','Tx_by_WS','Ty_by_WS','Tz_by_WS']

stimuli = [0,1,2,3,4,5]
stimuli_names = ['pitch up','pitch down','roll left','roll right','yaw left','yaw right']

In [ ]:
'''
    Design parameterts and other variables
    
    fs          : sampling frequency in Hz
    duration    : duration of the wing stroke in seconds, selected to be larger than the max spike timing across all trials (wing strokes) and subjects (moths)
    T           : duration of the wing stroke in number of samples
    N           : number of channels (muscles)
    sigma       : Gaussian kernel bandwidth in seconds
    omega       : test data set ratio
    no_modes    : number of retained PCA modes
    MCT         : number of Monte Carlo runs, associated with random train/test data set splits

'''

fs, duration = 10e3, 0.06 
T = int(fs*duration)
N = 10
sigma, omega, no_Modes = .0025, .15, 20
MCT = 10

In [ ]:
Z = [1,2,3,4,6,7,8,9,10] # subject (moth) indices
test_accuracy = {}

for index in Z:
    
    spike_times, targets, no_Trials = DataFormatting( path, index_database, index-1, all_neurons, stimuli )
    signals, _, _, targets = GaussianKernelFiltering( spike_times, targets, sum(no_Trials[0]), all_neurons, stimuli, duration, fs, sigma )
    
    signals = signals.reshape((signals.shape[0],signals.shape[1]*signals.shape[2]))
    rate = 0
    test_accuracy['moth-{}'.format(index)] = 0
    
    for m in range(MCT):
        
        signals_train, signals_test, targets_train, targets_test = train_test_split(signals, targets, test_size=omega)
        
        pca = PCA(n_components=no_Modes)
        pca.fit(signals_train)
        signals_train, signals_test  = pca.fit_transform(signals_train), pca.transform(signals_test)
        
        clf = LDA()
        clf.fit(signals_train, np.ravel(targets_train))            
        targets_predicted = np.expand_dims(clf.predict(signals_test), axis=1)
        rate = ( np.sum(targets_predicted == targets_test)/targets_test.shape[0] )
        test_accuracy['moth-{}'.format(index)] += rate/MCT
        
        print('  (m={}) Test rate for moth-{}: {:0.4f}'.format(m,index, rate ))
    
    